In [27]:
import sys
sys.path.append("/Users/kim/Desktop/repos/Philippines_Visitor")

import pandas as pd
from db_conn import get_engine

query = """
SELECT *
FROM (
    SELECT 
        c.country,
        y.year,
        y.total_visitors,
        RANK() OVER (
            PARTITION BY y.year 
            ORDER BY y.total_visitors DESC
        ) AS rank
    from yearly_visitors y
    JOIN countries_list c
        ON y.country_id = c.country_id
) AS ranked
WHERE rank <= 10
ORDER BY year, rank;
"""

def get_top10_per_year():
    engine = get_engine()
    df = pd.read_sql(query, engine)
    return df


top_10 = get_top10_per_year()
top_10.to_csv("Top10countries.csv")

In [26]:
top_10
# Count how many times each country appears
country_counts = top_10['country'].value_counts()

print(country_counts)

country
South Korea             11
Usa                     11
Japan                   11
China                   11
Canada                  11
United Kingdom          11
Australia               10
Singapore                9
Taiwan                   9
Malaysia                 9
India                    3
Germany                  1
Residences               1
Turkey                   1
United Arab Emirates     1
Name: count, dtype: int64


In [18]:
years = top_10['year'].unique()
countries = top_10['country'].unique()

full_index = pd.MultiIndex.from_product([years, countries], names=['year', 'country'])
df_full = top_10.set_index(['year', 'country']).reindex(full_index).reset_index()
df_full['total_visitors'] = df_full['total_visitors'].fillna(0)

df_full.to_csv("Full_TOP10.csv")

In [10]:
transposed = top_10.pivot_table(
        index="country",
        columns='year',
        values='total_visitors'
)

transposed.to_csv("TOP10_transposed.csv")
transposed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, Australia to Usa
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   2015    10 non-null     float64
 1   2016    10 non-null     float64
 2   2017    10 non-null     float64
 3   2018    10 non-null     float64
 4   2019    10 non-null     float64
 5   2020    10 non-null     float64
 6   2021    10 non-null     float64
 7   2022    10 non-null     float64
 8   2023    10 non-null     float64
 9   2024    10 non-null     float64
 10  2025    10 non-null     float64
dtypes: float64(11)
memory usage: 1.4+ KB


In [3]:
df_interpolated = transposed.interpolate(axis=1)  # interpolate along years
df_interpolated

country,Australia,Canada,China,Germany,India,Japan,Malaysia,Residences,Singapore,South Korea,Taiwan,Turkey,United Arab Emirates,United Kingdom,Usa
year,,,,,,,,,,,,,,,
2015,241187.0,156363.0,490841.0,4.924480e+05,4.940550e+05,495662.0,155814.00,168495.0,181176.00,1339678.0,177670.0,169976.333333,162282.666667,154589.0,779217.0
2016,251098.0,175631.0,675663.0,6.288547e+05,5.820463e+05,535238.0,139133.00,157595.0,176057.00,1475081.0,229303.0,210635.000000,191967.000000,173299.0,869463.0
2017,259433.0,200640.0,968447.0,8.403580e+05,7.122690e+05,584180.0,143566.00,156101.5,168637.00,1607821.0,236777.0,218754.000000,200731.000000,182708.0,957813.0
2018,279828.0,226446.0,1257962.0,1.049248e+06,8.405347e+05,631821.0,145246.00,158521.0,171796.00,1624251.0,242411.0,228622.000000,214833.000000,201044.0,1034471.0
2019,286170.0,238850.0,1743309.0,1.389802e+06,1.036295e+06,682788.0,139882.00,149238.5,158595.00,1989322.0,327273.0,287917.333333,248561.666667,209206.0,1064440.0
2020,55330.0,55273.0,170432.0,2.589300e+04,2.901400e+04,136664.0,187217.25,237770.5,288323.75,338877.0,48644.0,45756.000000,42868.000000,39980.0,211816.0
2021,NaN,6781.0,9674.0,8.438000e+03,7.202000e+03,15024.0,11796.00,8568.0,7512.00,6456.0,5415.0,4374.000000,2733.000000,4348.0,39326.0
2022,137974.0,121413.0,39627.0,4.558450e+04,5.154200e+04,99557.0,46805.00,50126.5,53448.00,428014.0,346269.0,264524.000000,182779.000000,101034.0,505089.0
2023,266551.0,221920.0,263836.0,2.777507e+05,2.916653e+05,305580.0,97639.00,123434.5,149230.00,1450858.0,194851.0,181466.666667,168082.333333,154698.0,903299.0


In [4]:
import bar_chart_race as bcr

bcr.bar_chart_race(
    df=df_interpolated,
    filename='Top 10 Countries Over Time.gif',          # optional: saves video
    title='Top Countries by Visitors Over Time',
    period_fmt='%Y',           # depends on your index format
    steps_per_period=50,
    period_length=500
)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/opt/homebrew/anaconda3/lib/python3.9/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
/opt/homebrew/anaconda3/lib/python3.9/site-packages/bar_chart_race/_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
MovieWriter imagemagick unavailable; using Pillow instead.
